In [5]:
using ForwardDiff, GenericSVD, LinearAlgebra

## Singular Value Decomposition -- Laguerre Ensembles
$$ dA = \prod_{i < j} |\sigma_i^2 - \sigma_j^2| \prod_i\sigma_i^{m-n}(H'dU)^\wedge(d\Sigma)^\wedge (V'dV)^\wedge $$

In [2]:
function svd_theory(A) #  |Π(σᵢ²-σⱼ²) Πσᵢ^(m-n)| 
    σ = svdvals(A)
    m,n = size(A)
    abs(prod([σ[i]^2-σ[j]^2 for i=1:n, j=1:n if i<j])*prod(σ)^(m-n))   
end

svd_theory (generic function with 1 method)

In [3]:
function svd_auto(A)
    H,  σ, V = svd(A,thin=false)
    dU,dσ,dV = (ForwardDiff.jacobian(x -> svd(x)[i], A) for i=1:3)    
    HᵀdU = kron(eye(V), H') * dU   
    VᵀdV = kron(eye(V), V') * dV    
    rowsU = vec((tril(ones(A),-1))) .==1
    rowsV = vec((tril(ones(V),-1))) .==1   
    HᵀdU = HᵀdU[rowsU, :]; VᵀdV = VᵀdV[rowsV, :]   
    1/abs(det( [dσ; HᵀdU; VᵀdV] )) 
end

svd_auto (generic function with 1 method)

In [6]:
# Experiment vs Theory (m≥n)
A = randn(6,4)
svd_auto(A), svd_theory(A)

(149824.04262999113, 149824.04262999224)